# 점포 전처리
* 서울시 공공데이터 포털에서 '서울특별시 서대문구 휴게음식점 인허가 정보' 다운
* 서울시 공공데이터 포털에서 '서울특별시 서대문구 제과점영업 인허가 정보' 다운
* 영업상태가 영업중인 점포만 골라내기
* 필요한 컬럼만 추출(사업장명, 도로명주소, 업태구분명, 좌표정보(X), 좌표정보(Y))
* 좌표 표시 방법 변경
* '1. 점포_전처리.xlsx'파일 생성
 * 업태구분명이 기타 휴게음식점인 곳까지 포함시켜서 새로운 sheet에 카페, 편의점별로 정리(수작업)

In [1]:
import pandas as pd
import numpy as np
from pyproj import Proj, transform
from tqdm import tqdm

## 카페, 편의점

In [2]:
url = './데이터 원본/서울특별시 서대문구 휴게음식점 인허가 정보.csv'
df = pd.read_csv(url, encoding='cp949')
cafe_conv_df = df.copy()

In [3]:
# 영업 중인 점포
cafe_conv_df = cafe_conv_df[cafe_conv_df['영업상태명'] == '영업/정상']
cafe_conv_df

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
22,3120000,3120000-104-1970-08418,19701001,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,3120000,3120000-104-1970-08500,19701218,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,3120000,3120000-104-1972-09893,19720303,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213,3120000,3120000-104-1983-09883,19831109,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,69.01,NaN,NaN,NaN
247,3120000,3120000-104-1984-10537,19840817,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4288,3120000,3120000-104-2021-00139,20210820,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4289,3120000,3120000-104-2021-00140,20210823,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4290,3120000,3120000-104-2021-00141,20210826,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4291,3120000,3120000-104-2021-00142,20210826,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# 필요한 컬럼
col = ['사업장명', '도로명주소', '업태구분명', '좌표정보(X)', '좌표정보(Y)']
cafe_conv_df = cafe_conv_df.loc[:, col]
cafe_conv_df

,사업장명,도로명주소,업태구분명,좌표정보(X),좌표정보(Y)
22,명,서울특별시 서대문구 통일로 107-15 (미근동(지하1층)),다방,197035.204094,451356.548632
25,호수,서울특별시 서대문구 수색로 28-3 (남가좌동),다방,192458.345686,451900.763367
41,학,서울특별시 서대문구 연희로 267 (홍은동),다방,194268.239983,453186.156136
213,파파존스피자,서울특별시 서대문구 성산로 379 (연희동),패스트푸드,193840.053216,451216.225488
247,사랑커피숍,서울특별시 서대문구 응암로 138 세일빌딩 지하1층 (북가좌동),다방,192423.411792,453573.217342
...,...,...,...,...,...
4288,호두앤,서울특별시 서대문구 신촌로 83 현대백화점신촌점 지하1층 (창천동),백화점,194265.067640,450433.691021
4289,포케올데이 연희점,서울특별시 서대문구 연희로15길 39 1층 (연희동),일반조리판매,193743.603949,451871.793129
4290,올리(ollie),서울특별시 서대문구 모래내로 364 1층 (홍은동),커피숍,194323.087614,453288.060643
4291,스윗드로우 케이크(Sweetdraw cake),서울특별시 서대문구 신촌역로 22-5 신촌박스퀘어 2층 37호 (대현동),기타 휴게음식점,194955.406354,450620.463749


In [5]:
# 결측치 제거
cafe_conv_df = cafe_conv_df.dropna()
cafe_conv_df

,사업장명,도로명주소,업태구분명,좌표정보(X),좌표정보(Y)
22,명,서울특별시 서대문구 통일로 107-15 (미근동(지하1층)),다방,197035.204094,451356.548632
25,호수,서울특별시 서대문구 수색로 28-3 (남가좌동),다방,192458.345686,451900.763367
41,학,서울특별시 서대문구 연희로 267 (홍은동),다방,194268.239983,453186.156136
213,파파존스피자,서울특별시 서대문구 성산로 379 (연희동),패스트푸드,193840.053216,451216.225488
247,사랑커피숍,서울특별시 서대문구 응암로 138 세일빌딩 지하1층 (북가좌동),다방,192423.411792,453573.217342
...,...,...,...,...,...
4288,호두앤,서울특별시 서대문구 신촌로 83 현대백화점신촌점 지하1층 (창천동),백화점,194265.067640,450433.691021
4289,포케올데이 연희점,서울특별시 서대문구 연희로15길 39 1층 (연희동),일반조리판매,193743.603949,451871.793129
4290,올리(ollie),서울특별시 서대문구 모래내로 364 1층 (홍은동),커피숍,194323.087614,453288.060643
4291,스윗드로우 케이크(Sweetdraw cake),서울특별시 서대문구 신촌역로 22-5 신촌박스퀘어 2층 37호 (대현동),기타 휴게음식점,194955.406354,450620.463749


### 좌표 변환

In [6]:
proj_1 = Proj('+proj=tmerc +lat_0=38 +lon_0=127.0028902777778 +k=1 +x_0=200000 +y_0=500000 +ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43')
proj_2 = Proj(init='epsg:4326')

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:306: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))


In [7]:
cafe_conv_df = cafe_conv_df.rename(columns={"좌표정보(X)": "x", "좌표정보(Y)": 'y'})
cafe_conv_df

,사업장명,도로명주소,업태구분명,x,y
22,명,서울특별시 서대문구 통일로 107-15 (미근동(지하1층)),다방,197035.204094,451356.548632
25,호수,서울특별시 서대문구 수색로 28-3 (남가좌동),다방,192458.345686,451900.763367
41,학,서울특별시 서대문구 연희로 267 (홍은동),다방,194268.239983,453186.156136
213,파파존스피자,서울특별시 서대문구 성산로 379 (연희동),패스트푸드,193840.053216,451216.225488
247,사랑커피숍,서울특별시 서대문구 응암로 138 세일빌딩 지하1층 (북가좌동),다방,192423.411792,453573.217342
...,...,...,...,...,...
4288,호두앤,서울특별시 서대문구 신촌로 83 현대백화점신촌점 지하1층 (창천동),백화점,194265.067640,450433.691021
4289,포케올데이 연희점,서울특별시 서대문구 연희로15길 39 1층 (연희동),일반조리판매,193743.603949,451871.793129
4290,올리(ollie),서울특별시 서대문구 모래내로 364 1층 (홍은동),커피숍,194323.087614,453288.060643
4291,스윗드로우 케이크(Sweetdraw cake),서울특별시 서대문구 신촌역로 22-5 신촌박스퀘어 2층 37호 (대현동),기타 휴게음식점,194955.406354,450620.463749


In [8]:
cafe_conv_df['x']

22      197035.204094
25      192458.345686
41      194268.239983
213     193840.053216
247     192423.411792
            ...      
4288    194265.067640
4289    193743.603949
4290    194323.087614
4291    194955.406354
4292    195474.704574
Name: x, Length: 1110, dtype: float64

In [9]:
cafe_conv_df['y']

22      451356.548632
25      451900.763367
41      453186.156136
213     451216.225488
247     453573.217342
            ...      
4288    450433.691021
4289    451871.793129
4290    453288.060643
4291    450620.463749
4292    455263.438956
Name: y, Length: 1110, dtype: float64

In [10]:
DataFrame = cafe_conv_df.copy()

x_list = []
y_list = []

for idx, row in tqdm(DataFrame.iterrows()):
    x, y = row['x'], row['y']
    x_, y_ = transform(proj_1, proj_2, x, y)
    x_list.append(x_)
    y_list.append(y_)
    
cafe_conv_df['경도'] = x_list
cafe_conv_df['위도'] = y_list

0it [00:00, ?it/s]<ipython-input-10-020e0a774d77>:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x_, y_ = transform(proj_1, proj_2, x, y)
1110it [00:59, 18.70it/s]


In [11]:
cafe_conv_df

,사업장명,도로명주소,업태구분명,x,y,경도,위도
22,명,서울특별시 서대문구 통일로 107-15 (미근동(지하1층)),다방,197035.204094,451356.548632,126.967231,37.564485
25,호수,서울특별시 서대문구 수색로 28-3 (남가좌동),다방,192458.345686,451900.763367,126.915421,37.569362
41,학,서울특별시 서대문구 연희로 267 (홍은동),다방,194268.239983,453186.156136,126.935898,37.580956
213,파파존스피자,서울특별시 서대문구 성산로 379 (연희동),패스트푸드,193840.053216,451216.225488,126.931067,37.563205
247,사랑커피숍,서울특별시 서대문구 응암로 138 세일빌딩 지하1층 (북가좌동),다방,192423.411792,453573.217342,126.915009,37.584430
...,...,...,...,...,...,...,...
4288,호두앤,서울특별시 서대문구 신촌로 83 현대백화점신촌점 지하1층 (창천동),백화점,194265.067640,450433.691021,126.935884,37.556157
4289,포케올데이 연희점,서울특별시 서대문구 연희로15길 39 1층 (연희동),일반조리판매,193743.603949,451871.793129,126.929970,37.569111
4290,올리(ollie),서울특별시 서대문구 모래내로 364 1층 (홍은동),커피숍,194323.087614,453288.060643,126.936518,37.581875
4291,스윗드로우 케이크(Sweetdraw cake),서울특별시 서대문구 신촌역로 22-5 신촌박스퀘어 2층 37호 (대현동),기타 휴게음식점,194955.406354,450620.463749,126.943695,37.557844


In [12]:
# 인덱스 정리
cafe_conv_df.index = range(len(cafe_conv_df))
cafe_conv_df

,사업장명,도로명주소,업태구분명,x,y,경도,위도
0,명,서울특별시 서대문구 통일로 107-15 (미근동(지하1층)),다방,197035.204094,451356.548632,126.967231,37.564485
1,호수,서울특별시 서대문구 수색로 28-3 (남가좌동),다방,192458.345686,451900.763367,126.915421,37.569362
2,학,서울특별시 서대문구 연희로 267 (홍은동),다방,194268.239983,453186.156136,126.935898,37.580956
3,파파존스피자,서울특별시 서대문구 성산로 379 (연희동),패스트푸드,193840.053216,451216.225488,126.931067,37.563205
4,사랑커피숍,서울특별시 서대문구 응암로 138 세일빌딩 지하1층 (북가좌동),다방,192423.411792,453573.217342,126.915009,37.584430
...,...,...,...,...,...,...,...
1105,호두앤,서울특별시 서대문구 신촌로 83 현대백화점신촌점 지하1층 (창천동),백화점,194265.067640,450433.691021,126.935884,37.556157
1106,포케올데이 연희점,서울특별시 서대문구 연희로15길 39 1층 (연희동),일반조리판매,193743.603949,451871.793129,126.929970,37.569111
1107,올리(ollie),서울특별시 서대문구 모래내로 364 1층 (홍은동),커피숍,194323.087614,453288.060643,126.936518,37.581875
1108,스윗드로우 케이크(Sweetdraw cake),서울특별시 서대문구 신촌역로 22-5 신촌박스퀘어 2층 37호 (대현동),기타 휴게음식점,194955.406354,450620.463749,126.943695,37.557844


## 베이커리

In [13]:
url = './데이터 원본/서울특별시 서대문구 제과점영업 인허가 정보.csv'
bakeray_df = pd.read_csv(url, encoding='cp949')
bakeray_df

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,3120000,3120000-121-1967-00001,19671010,NaN,3,폐업,2,폐업,20130418.0,NaN,...,NaN,NaN,NaN,NaN,NaN,N,79.45,NaN,NaN,NaN
1,3120000,3120000-121-1971-00001,19710809,NaN,3,폐업,2,폐업,20150629.0,NaN,...,NaN,NaN,NaN,NaN,NaN,N,29.40,NaN,NaN,NaN
2,3120000,3120000-121-1973-00001,19731017,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3120000,3120000-121-1976-00001,19760228,NaN,3,폐업,2,폐업,20100813.0,NaN,...,NaN,NaN,NaN,NaN,NaN,N,240.30,NaN,NaN,NaN
4,3120000,3120000-121-1977-00001,19770325,NaN,3,폐업,2,폐업,20120829.0,NaN,...,NaN,NaN,NaN,NaN,NaN,N,19.17,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,3120000,3120000-121-2021-00002,20210223,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
367,3120000,3120000-121-2021-00003,20210426,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,3120000,3120000-121-2021-00004,20210625,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,3120000,3120000-121-2021-00005,20210713,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# 영업 중인 점포
bakeray_df = bakeray_df[bakeray_df['영업상태명'] == '영업/정상']
bakeray_df

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
2,3120000,3120000-121-1973-00001,19731017,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3120000,3120000-121-1979-00001,19790627,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,97.29,NaN,NaN,NaN
6,3120000,3120000-121-1980-00001,19801209,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,82.45,NaN,NaN,NaN
10,3120000,3120000-121-1981-00003,19810502,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,37.80,NaN,NaN,NaN
59,3120000,3120000-121-1999-00005,19990811,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,3120000,3120000-121-2021-00002,20210223,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
367,3120000,3120000-121-2021-00003,20210426,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,3120000,3120000-121-2021-00004,20210625,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,3120000,3120000-121-2021-00005,20210713,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# 필요한 컬럼
col = ['사업장명', '도로명주소', '업태구분명', '좌표정보(X)', '좌표정보(Y)']
bakeray_df = bakeray_df.loc[:, col]
bakeray_df

,사업장명,도로명주소,업태구분명,좌표정보(X),좌표정보(Y)
2,주재근베이커리 홍제,서울특별시 서대문구 통일로 438 1층지하1층 (홍제동),제과점영업,195101.203770,453985.399133
5,이화당,서울특별시 서대문구 성산로 521 (대신동),제과점영업,194956.054121,451227.707140
6,피터팬,서울특별시 서대문구 증가로 10 (연희동),제과점영업,193900.025384,451904.382481
10,마실중(ing),서울특별시 서대문구 모래내로 357 (홍은동),제과점영업,194248.499509,453271.502182
59,파리바게뜨 아현점,서울특별시 서대문구 신촌로35길 10 402엠동 1층 112호 (북아현동 이편한세상신촌),제과점영업,196104.357264,450747.786158
...,...,...,...,...,...
366,글룩베이커리,서울특별시 서대문구 명지대7길 19 1층 (남가좌동),제과점영업,193050.383068,453504.344166
367,뚜레쥬르 충암점,서울특별시 서대문구 가좌로 141 1층 (홍은동),제과점영업,193183.651737,453583.312173
368,더파이홀2,서울특별시 서대문구 연세로5나길 18 3층 (창천동),제과점영업,194201.714525,450534.090429
369,달다구리 제작소,서울특별시 서대문구 통일로27길 5 1층 3호 (홍제동),제과점영업,195432.285773,453467.338107


In [16]:
# 결측치 제거
bakeray_df = bakeray_df.dropna()
bakeray_df

,사업장명,도로명주소,업태구분명,좌표정보(X),좌표정보(Y)
2,주재근베이커리 홍제,서울특별시 서대문구 통일로 438 1층지하1층 (홍제동),제과점영업,195101.203770,453985.399133
5,이화당,서울특별시 서대문구 성산로 521 (대신동),제과점영업,194956.054121,451227.707140
6,피터팬,서울특별시 서대문구 증가로 10 (연희동),제과점영업,193900.025384,451904.382481
10,마실중(ing),서울특별시 서대문구 모래내로 357 (홍은동),제과점영업,194248.499509,453271.502182
59,파리바게뜨 아현점,서울특별시 서대문구 신촌로35길 10 402엠동 1층 112호 (북아현동 이편한세상신촌),제과점영업,196104.357264,450747.786158
...,...,...,...,...,...
366,글룩베이커리,서울특별시 서대문구 명지대7길 19 1층 (남가좌동),제과점영업,193050.383068,453504.344166
367,뚜레쥬르 충암점,서울특별시 서대문구 가좌로 141 1층 (홍은동),제과점영업,193183.651737,453583.312173
368,더파이홀2,서울특별시 서대문구 연세로5나길 18 3층 (창천동),제과점영업,194201.714525,450534.090429
369,달다구리 제작소,서울특별시 서대문구 통일로27길 5 1층 3호 (홍제동),제과점영업,195432.285773,453467.338107


In [17]:
bakeray_df = bakeray_df.rename(columns={"좌표정보(X)": "x", "좌표정보(Y)": 'y'})
bakeray_df

,사업장명,도로명주소,업태구분명,x,y
2,주재근베이커리 홍제,서울특별시 서대문구 통일로 438 1층지하1층 (홍제동),제과점영업,195101.203770,453985.399133
5,이화당,서울특별시 서대문구 성산로 521 (대신동),제과점영업,194956.054121,451227.707140
6,피터팬,서울특별시 서대문구 증가로 10 (연희동),제과점영업,193900.025384,451904.382481
10,마실중(ing),서울특별시 서대문구 모래내로 357 (홍은동),제과점영업,194248.499509,453271.502182
59,파리바게뜨 아현점,서울특별시 서대문구 신촌로35길 10 402엠동 1층 112호 (북아현동 이편한세상신촌),제과점영업,196104.357264,450747.786158
...,...,...,...,...,...
366,글룩베이커리,서울특별시 서대문구 명지대7길 19 1층 (남가좌동),제과점영업,193050.383068,453504.344166
367,뚜레쥬르 충암점,서울특별시 서대문구 가좌로 141 1층 (홍은동),제과점영업,193183.651737,453583.312173
368,더파이홀2,서울특별시 서대문구 연세로5나길 18 3층 (창천동),제과점영업,194201.714525,450534.090429
369,달다구리 제작소,서울특별시 서대문구 통일로27길 5 1층 3호 (홍제동),제과점영업,195432.285773,453467.338107


In [18]:
DataFrame = bakeray_df.copy()

x_list = []
y_list = []

for idx, row in tqdm(DataFrame.iterrows()):
    x, y = row['x'], row['y']
    x_, y_ = transform(proj_1, proj_2, x, y)
    x_list.append(x_)
    y_list.append(y_)
    
bakeray_df['경도'] = x_list
bakeray_df['위도'] = y_list

0it [00:00, ?it/s]<ipython-input-18-cb3223628036>:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x_, y_ = transform(proj_1, proj_2, x, y)
113it [00:06, 17.57it/s]


In [19]:
bakeray_df.index = range(len(bakeray_df))
bakeray_df

,사업장명,도로명주소,업태구분명,x,y,경도,위도
0,주재근베이커리 홍제,서울특별시 서대문구 통일로 438 1층지하1층 (홍제동),제과점영업,195101.203770,453985.399133,126.945323,37.588162
1,이화당,서울특별시 서대문구 성산로 521 (대신동),제과점영업,194956.054121,451227.707140,126.943699,37.563315
2,피터팬,서울특별시 서대문구 증가로 10 (연희동),제과점영업,193900.025384,451904.382481,126.931740,37.569405
3,마실중(ing),서울특별시 서대문구 모래내로 357 (홍은동),제과점영업,194248.499509,453271.502182,126.935674,37.581725
4,파리바게뜨 아현점,서울특별시 서대문구 신촌로35길 10 402엠동 1층 112호 (북아현동 이편한세상신촌),제과점영업,196104.357264,450747.786158,126.956698,37.558997
...,...,...,...,...,...,...,...
108,글룩베이커리,서울특별시 서대문구 명지대7길 19 1층 (남가좌동),제과점영업,193050.383068,453504.344166,126.922108,37.583815
109,뚜레쥬르 충암점,서울특별시 서대문구 가좌로 141 1층 (홍은동),제과점영업,193183.651737,453583.312173,126.923616,37.584527
110,더파이홀2,서울특별시 서대문구 연세로5나길 18 3층 (창천동),제과점영업,194201.714525,450534.090429,126.935166,37.557061
111,달다구리 제작소,서울특별시 서대문구 통일로27길 5 1층 3호 (홍제동),제과점영업,195432.285773,453467.338107,126.949075,37.583496


## 카페, 편의점, 베이커리 합치기

In [20]:
store_df = cafe_conv_df.copy()
store_df = store_df.append(bakeray_df)
store_df

,사업장명,도로명주소,업태구분명,x,y,경도,위도
0,명,서울특별시 서대문구 통일로 107-15 (미근동(지하1층)),다방,197035.204094,451356.548632,126.967231,37.564485
1,호수,서울특별시 서대문구 수색로 28-3 (남가좌동),다방,192458.345686,451900.763367,126.915421,37.569362
2,학,서울특별시 서대문구 연희로 267 (홍은동),다방,194268.239983,453186.156136,126.935898,37.580956
3,파파존스피자,서울특별시 서대문구 성산로 379 (연희동),패스트푸드,193840.053216,451216.225488,126.931067,37.563205
4,사랑커피숍,서울특별시 서대문구 응암로 138 세일빌딩 지하1층 (북가좌동),다방,192423.411792,453573.217342,126.915009,37.584430
...,...,...,...,...,...,...,...
108,글룩베이커리,서울특별시 서대문구 명지대7길 19 1층 (남가좌동),제과점영업,193050.383068,453504.344166,126.922108,37.583815
109,뚜레쥬르 충암점,서울특별시 서대문구 가좌로 141 1층 (홍은동),제과점영업,193183.651737,453583.312173,126.923616,37.584527
110,더파이홀2,서울특별시 서대문구 연세로5나길 18 3층 (창천동),제과점영업,194201.714525,450534.090429,126.935166,37.557061
111,달다구리 제작소,서울특별시 서대문구 통일로27길 5 1층 3호 (홍제동),제과점영업,195432.285773,453467.338107,126.949075,37.583496


In [21]:
store_df.index = range(len(store_df))
store_df

,사업장명,도로명주소,업태구분명,x,y,경도,위도
0,명,서울특별시 서대문구 통일로 107-15 (미근동(지하1층)),다방,197035.204094,451356.548632,126.967231,37.564485
1,호수,서울특별시 서대문구 수색로 28-3 (남가좌동),다방,192458.345686,451900.763367,126.915421,37.569362
2,학,서울특별시 서대문구 연희로 267 (홍은동),다방,194268.239983,453186.156136,126.935898,37.580956
3,파파존스피자,서울특별시 서대문구 성산로 379 (연희동),패스트푸드,193840.053216,451216.225488,126.931067,37.563205
4,사랑커피숍,서울특별시 서대문구 응암로 138 세일빌딩 지하1층 (북가좌동),다방,192423.411792,453573.217342,126.915009,37.584430
...,...,...,...,...,...,...,...
1218,글룩베이커리,서울특별시 서대문구 명지대7길 19 1층 (남가좌동),제과점영업,193050.383068,453504.344166,126.922108,37.583815
1219,뚜레쥬르 충암점,서울특별시 서대문구 가좌로 141 1층 (홍은동),제과점영업,193183.651737,453583.312173,126.923616,37.584527
1220,더파이홀2,서울특별시 서대문구 연세로5나길 18 3층 (창천동),제과점영업,194201.714525,450534.090429,126.935166,37.557061
1221,달다구리 제작소,서울특별시 서대문구 통일로27길 5 1층 3호 (홍제동),제과점영업,195432.285773,453467.338107,126.949075,37.583496


In [22]:
# store_df.to_excel('1. 점포_전처리.xlsx')

'업태구분명'이 기타 휴게음식점인 곳 중에서도 카페나 편의점이 있음.
이런 점포들도 고려하기 위해 엑셀 파일을 직접 보며 걸러내기(sheet별로 카페, 편의점 따로 만들기)